# LAB | Two Sample Hypothesis Tests

### Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [42]:
# import numpy and pandas
import pandas as pd
import numpy as np
from scipy.stats import trim_mean, mode, skew, gaussian_kde, pearsonr, spearmanr, beta
from statsmodels.stats.weightstats import ztest as ztest

from scipy.stats import ttest_ind, norm, t
from scipy.stats import f_oneway
from scipy.stats import sem
from scipy import stats
import math
import scipy.stats as st

### t-statistic: your “signal-to-noise” number:
t ≈ 0 → the group means are basically the same (difference is tiny compared to the noise).

Large |t| (positive or negative) → the means are many SEs apart, much larger than chance would explain.

Sign:
- Positive = the first group’s mean > second group’s mean.
- Negative = the first group’s mean < second group’s mean.

### with p-value: 
Big |t|, small p → reject H₀ (means are different).

Small |t|, big p → fail to reject H₀ (no evidence of difference).

## Challenge 1 - Independent Sample T-tests

In this challenge, we will be using the Pokemon dataset. Before applying statistical methods to this data, let's first examine the data.

To load the data, run the code below.

In [7]:
# Run this code:

pokemon = pd.read_csv('../data/pokemon.csv')

Let's start off by looking at the `head` function in the cell below.

In [24]:
pokemon.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


The first thing we would like to do is compare the legendary Pokemon to the regular Pokemon. To do this, we should examine the data further. What is the count of legendary vs. non legendary Pokemons?

In [37]:
legendary = pokemon[pokemon["Legendary"] == True]
regular   = pokemon[pokemon["Legendary"] == False]

print("Legendary:", legendary.shape[0])
print("Regular:", regular.shape[0])

Legendary: 65
Regular: 735


Compute the mean and standard deviation of the total points for both legendary and non-legendary Pokemon.

In [40]:
# means & stds
print("Legendary mean, Total points:", legendary["Total"].mean())
print("Legendary std:",  legendary["Total"].std())
print("Regular mean, Total points:",   regular["Total"].mean())
print("Regular std:",    regular["Total"].std())

Legendary mean, Total points: 637.3846153846154
Legendary std: 60.93738905315344
Regular mean, Total points: 417.21360544217686
Regular std: 106.76041745713005


The computation of the mean might give us a clue regarding how the statistical test may turn out; However, it certainly does not prove whether there is a significant difference between the two groups.

In the cell below, use the `ttest_ind` function in `scipy.stats` to compare the the total points for legendary and non-legendary Pokemon. Since we do not have any information about the population, assume the variances are not equal.

In [44]:
t_stat, pvalue = st.ttest_ind(
    legendary["Total"].dropna(), 
    regular["Total"].dropna(),
      alternative='two-sided', 
      equal_var=False
      )
print("t-stat:", t_stat)
print("p-value:", pvalue)

t-stat: 25.8335743895517
p-value: 9.357954335957446e-47


What do you conclude from this test? Write your conclusions below.

In [ ]:
# Null hypothesis: the mean total points for Legendary and Regular are equal
# With a large t-statistic, and a p-value close to 0, we can reject the null hypothesis. 
# Statistically, the points are not the same, supporting the alternative (H1)

How about we try to compare the different types of pokemon? In the cell below, list the types of Pokemon from column `Type 1` and the count of each type.

In [53]:
print("Type 1 Pokemon:", pokemon["Type 1"].unique())

type_1 = pokemon["Type 1"]

print("Type 1 count:", type_1.shape[0])


Type 1 Pokemon: ['Grass' 'Fire' 'Water' 'Bug' 'Normal' 'Poison' 'Electric' 'Ground'
 'Fairy' 'Fighting' 'Psychic' 'Rock' 'Ghost' 'Ice' 'Dragon' 'Dark' 'Steel'
 'Flying']
Type 1 count: 800


Since water is the largest group of Pokemon, compare the mean and standard deviation of water Pokemon to all other Pokemon.

In [ ]:
water = pokemon[pokemon["Type 1"] == "Water"]
others = pokemon[pokemon["Type 1"] != "Water"]

print("Type 1 Water Total std:", water["Total"].std())
print("Type 1 Others Total std:", others["Total"].std())
print("Type 1 Water Total mean:", water["Total"].mean())
print("Type 1 Others Total mean:", others["Total"].mean())


Type 1 Water Total std: 113.1882660643146
Type 1 Others Total std: 121.0916823020807
Type 1 Water Total mean: 430.45535714285717
Type 1 Others Total mean: 435.85901162790697


Perform a hypothesis test comparing the mean of total points for water Pokemon to all non-water Pokemon. Assume the variances are equal. 

In [68]:
t_stat, pvalue = st.ttest_ind(
    water["Total"].dropna(), 
    others["Total"].dropna(),
      alternative='two-sided', 
      equal_var=False
      )
print("t-stat:", t_stat)
print("p-value:", pvalue)

t-stat: -0.4638681676327303
p-value: 0.6433915385821449


Write your conclusion below.

In [ ]:
# Null hypothesis: the mean total points for Water and Others are not statistically different
# Given a negative t-stat, and a p-value > 0.05, we can accept the H0 hypothesis. 
# Conclusion: No statistical difference in Total points between Water and Others

# Challenge 2 - Matched Pairs Test

In this challenge we will compare dependent samples of data describing our Pokemon. Our goal is to see whether there is a significant difference between each Pokemon's defense and attack scores. Our hypothesis is that the defense and attack scores are equal. In the cell below, import the `ttest_rel` function from `scipy.stats` and compare the two columns to see if there is a statistically significant difference between them.

In [71]:
from scipy import stats

t_stat, p_val = stats.ttest_rel(pokemon["Attack"], pokemon["Defense"], axis=0, nan_policy='propagate', alternative='two-sided')

print("t-stat:", t_stat)
print("p-value:", p_val)


t-stat: 4.325566393330478
p-value: 1.7140303479358558e-05


Describe the results of the test in the cell below.

In [ ]:
# Null hypothesis: the mean scores for Attack and Defense are equal
# Given a large t-stat, and a p-value < 0.05, we can reject the H0 hypothesis. 
# Conclusion: Significant statistical difference in Attack and Defense scores points

t 4.325566393330478
p 1.7140303479358558e-05


We are also curious about whether there is a significant difference between the mean of special defense and the mean of special attack. Perform the hypothesis test in the cell below. 

In [84]:

from scipy import stats

t_stat, p_val = stats.ttest_rel(pokemon["Sp. Atk"],pokemon["Sp. Def"], axis=0, nan_policy='propagate', alternative='two-sided')

print("t-stat:", t_stat)
print("p-value:", p_val)


t-stat: 0.853986188453353
p-value: 0.3933685997548122


Describe the results of the test in the cell below.

In [ ]:
# H0: The mean Special Attack equals the mean Special Defense
# H1: The mean Special Attack does not equal the mean Special Defense

# Result: t = 0.85, p = 0.39
# Decision: p > 0.05 --> fail to reject H0
# Conclusion: There is no statistically significant difference between mean Special Attack and mean Special Defense.

As you may recall, a two sample matched pairs test can also be expressed as a one sample test of the difference between the two dependent columns.

Import the `ttest_1samp` function and perform a one sample t-test of the difference between defense and attack. Test the hypothesis that the difference between the means is zero. Confirm that the results of the test are the same.

In [ ]:
mean_defense = pokemon["Defense"].mean()
print("mean defense score:", mean_defense)
mean_attack = pokemon["Attack"].mean()
print("mean attack score:", mean_attack)

t_stat, p_val = stats.ttest_1samp(pokemon["Defense"], popmean=79, alternative="two-sided")

print("t-stat:", t_stat)
print("p-value:", p_val)


# Result: t = -4.6, p < 0.05
# Decision: Reject H0
# Conclusion: There is a statistically significant difference between mean Attack and mean Defense.


mean defense score: 73.8425
mean attack score: 79.00125
t-stat: -4.677990807389497
p-value: 3.4011592999176298e-06


# Bonus Challenge - The Chi-Square Test

The Chi-Square test is used to determine whether there is a statistically significant difference in frequencies. In other words, we are testing whether there is a relationship between categorical variables or rather when the variables are independent. This test is an alternative to Fisher's exact test and is used in scenarios where sample sizes are larger. However, with a large enough sample size, both tests produce similar results. Read more about the Chi Squared test [here](https://en.wikipedia.org/wiki/Chi-squared_test).

In the cell below, create a contingency table using `pd.crosstab` comparing whether a Pokemon is legenadary or not and whether the Type 1 of a Pokemon is water or not.

In [98]:
table = pd.crosstab(pokemon["Legendary"] == True, pokemon["Type 1"] == "Water")

table

Type 1,False,True
Legendary,,
False,627,108
True,61,4


Perform a chi-squared test using the `chi2_contingency` function in `scipy.stats`. You can read the documentation of the function [here](https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.stats.chi2_contingency.html).

In [103]:
from scipy.stats import chi2_contingency

chi2, p, dof, expected = chi2_contingency(table)

print("chi2:", chi2)
print("p-value", p)
print("degrees of freedom:", dof)
print("expected", expected)


chi2: 2.9429200762850503
p-value 0.08625467249550985
degrees of freedom: 1
expected [[632.1 102.9]
 [ 55.9   9.1]]


Based on a 95% confidence, should we reject the null hypothesis?

In [ ]:
# H0: The Legnedary status and Water type are independent
# H1: Legendary and Water type are not independent 

# Result: chi12 = 2.94, p = 0.8
# Decision: p > 0.05 --> fail to reject H0
# Conclusion: There is no statistically significant difference between Legendary status and Water type